# RDD
Spark对数据的核心抽象：RDD(Resilient Distributed Dataset)弹性分布式数据集，分布式的元素集合。

#### 总的来说，每个spark程序都会按如下方式工作：
1. 从外部数据创建出输入RDD（或者在程序中对一个集合并行化）
2. 使用诸如filter()这样的转化操作对RDD进行转化，以定义新的RDD
3. 告诉Spark对需要重用的中间结果RDD执行persist()操作
4. 使用行动操作(例如count()和first()等)来触发一次并行计算，Spark会对计算进行优化后再执行

In [1]:
import pyspark

from pyspark import SparkConf,SparkContext

## 1.配置
先创建一个SparkConf对象来配置应用，然后基于这个SparkConf创建一个SparkContext对象。

关闭Spark可以调用SparkContext的stop()方法，或者直接退出应用（sys.exit()）

In [2]:
conf = SparkConf().setMaster("local").setAppName("My APP")
#只需要两个参数，一个集群URL，这里是local，这个特殊值可以单机运行
#一个应用名，链接到一个集群时，可以在集群管理器的用户界面看到。
sc = SparkContext(conf = conf)

lines = sc.textFile('file:///Users/fire/jupyter/data/ball2018.txt')
#使用相对路径时，系统默认是从hdfs://localhost:9000/目录下读取 文件的，
#但是 文件并不在这一目录下，所以sc.textFile()必须使用绝对路径，
lines

file:///Users/fire/jupyter/data/ball2018.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

## 2.创建RDD

In [3]:
# 1中展示了从外部读取数据来创建RDD，这里展示直接对一个集合并行化。
lines2 = sc.parallelize(["pandas","i like pandas"])
lines2.first()

'pandas'

## 3.RDD操作
RDD支持两类操作：
* 转化: 返回一个新的RDD。比如map(),filter()。
    
* 行动: 向驱动器程序返回结果或者把结果写入外部系统，会触发实际的计算。比如count(),first().

##### NOTE
1. 转化操作都是惰性求值的（类似tf的计算图），但可以随时运行一个行动操作强制执行。
2. 每当调用一个新的行动操作时，整个RDD都会从头开始计算。避免这种行为，将中间结果持久化。
3. RDD.unpersist()方法可以手动把持久化的RDD从缓存移除。

In [4]:
linespy = lines.filter(lambda line:"2018011" in line)
linespy

PythonRDD[4] at RDD at PythonRDD.scala:49

In [5]:
linespy.first()

'2018011 2018.01.25  03102123273311  336871794   8注7356371元  138注170751元 奖池:4亿   详情走势'

In [6]:
#RDD.persist() 缓存下来以持久化重用
linespy.persist()
#linespy.count() 报错 先不管了

PythonRDD[4] at RDD at PythonRDD.scala:49

In [10]:
#### 计算RDD各值平方 

#单机本地会报错 和count一样
# nums = sc.parallelize([1,2,3,4])
# squred = nums.map(lambda x:x*x).collect()
# for nums in squred:
#     print(nums)

In [11]:
#### 将数据切分为单词
lines3 = sc.parallelize(["hello world","hi"])
words = lines3.flatMap(lambda line: line.split(" "))
words.first()

'hello'

# 4.向RDD传递函数
注意！会把函数所在对象也序列化传出去，所以引用类里的函数，不要直接return处理self.x的，先赋值x = self.x再处理。

In [7]:
def contains(s):
    return "2018011" in s
word = lines.filter(contains)
word

PythonRDD[6] at RDD at PythonRDD.scala:49